In [1]:
# 使用reduce逐个合并DataFrame
from functools import reduce
import gc
from get_shp_path import get_shp_files

## 计算一批新无人机的VI
from qgis.core import (
    QgsApplication,
    QgsVectorLayer,
    QgsRasterLayer,
    QgsCoordinateReferenceSystem,
    QgsRasterInterface,
    Qgis,
    QgsFeedback,
    QgsProject,
    QgsField,
    QgsVariantUtils,
    QgsVectorDataProvider
)
from qgis.analysis import QgsZonalStatistics
import pandas as pd
import os
from tqdm import tqdm

QgsApplication.setPrefixPath(r'D:\software\tool_gis\QGIS 3.34.12\apps\qgis-ltr', True)
qgs = QgsApplication([], False)
qgs.initQgis()

def get_band(polygon_file_path,raster_file_path,band_name):

    polygon_layer = QgsVectorLayer(polygon_file_path, 'Polygons', 'ogr')
    raster_layer = QgsRasterLayer(raster_file_path, 'Raster')

    if not polygon_layer.isValid() or not raster_layer.isValid():
        raise ValueError("无法加载图层，请检查文件路径和格式。")

    # 计算多边形内的栅格平均值
    zonal_stats = QgsZonalStatistics(
        polygon_layer,
        raster_layer,
        stats=QgsZonalStatistics.Mean,
        rasterBand=1
    )
    zonal_stats.calculateStatistics(None)

    # 检索有关属性的信息
    # for field in polygon_layer.fields():
    #     print(field.name(), field.typeName())

    data = []
    features = polygon_layer.getFeatures()
    for feature in features:
        # print (feature.attributes())
        # print(f"Feature ID: {feature.id()}, Raster_mea: {feature["mean"]}" )
        data.append([feature['plot_id'],feature["mean"]])

    # 删除mean属性表 
    caps = polygon_layer.dataProvider().capabilities()
    # Check if a particular capability is supported:
    if caps & QgsVectorDataProvider.DeleteAttributes:
        # print('The layer supports DeleteAttributes')
        res = polygon_layer.dataProvider().deleteAttributes([3])
    return data

def batch_get_band(raster_folder_path, polygon_file_path, time):
    multiband_name = ['Blue','Green','NIR','Red','RedEdge']
    # 栅格文件的扩展名是.tif
    raster_file_extension = '.tif'
    # 存储所有波段的数据
    all_data = {}
    # 首先获取所有ID
    all_ids = set()
    
    for band_name in tqdm(multiband_name,desc=f'正在处理{time}的栅格数据'):
        raster_file_path = os.path.join(raster_folder_path, f"{band_name}{raster_file_extension}")
        try:
            data = get_band(polygon_file_path, raster_file_path, band_name)
            print(f"{band_name}完成")
            
            # 存储数据和收集所有唯一的ID
            all_data[band_name] = {row[0]: row[1] for row in data}
            all_ids.update(id for id, _ in data)
            
            # 清理内存
            del data
            gc.collect()
            
        except Exception as e:
            print(f"处理{band_name}时发生错误：{str(e)}")
            continue

    # 构建最终数据列表
    final_data = []
    for id in sorted(all_ids):  # 排序以保持一致的顺序
        row = [id]
        for band_name in multiband_name:
            row.append(all_data.get(band_name, {}).get(id, None))
        final_data.append(row)

    # 一次性创建DataFrame
    columns = ['id'] + multiband_name
    result_df = pd.DataFrame(final_data, columns=columns)
    
    # 优化数据类型
    result_df['id'] = result_df['id'].astype('int32')
    for band in multiband_name:
        if band in result_df.columns:
            result_df[band] = result_df[band].astype('float32')

    # 保存结果
    out_multiband_csv_filr_path = os.path.join(raster_folder_path, "multiband.csv")
    result_df.to_csv(out_multiband_csv_filr_path, index=False)
    
    # 清理内存
    del result_df, all_data, final_data
    gc.collect()

# 获取地址
polygon_file_path = r'D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\clip_shp\2024_plot_clip_shp'  # 替换为你的SHP文件路径
#0408有缺
l_time = ['0312','0401','0409','0412']
list_shp_files = get_shp_files(polygon_file_path)

for time in tqdm(l_time,desc='正在批量获取multiband'):
    raster_folder_path = rf'D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-24{time}' 
    for shp_file in list_shp_files:
        if time in shp_file:
            clip_shapefile = shp_file
            break
        else:
            clip_shapefile = rf"{polygon_file_path}\plot.shp"
    print(f"使用的是 {clip_shapefile} 进行裁剪")
    batch_get_band(raster_folder_path, polygon_file_path, time)

qgs.exitQgis()

正在批量获取multiband:   0%|          | 0/4 [00:00<?, ?it/s]

使用的是 D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\clip_shp\2024_plot_clip_shp\plot_240312.shp 进行裁剪


Blue完成


Green完成


NIR完成


...

Green完成


NIR完成


Red完成


正在批量获取multiband: 100%|██████████| 4/4 [00:52<00:00, 13.24s/it]

RedEdge完成


In [2]:
polygon_file_path = r'D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\clip_shp\2024_plot_clip_shp\plot_240312.shp'
raster_file_path = r'D:\work\DATA\DATA_TS4GPC\raw\UAV\CXZ-WN\CXZ-WN-2024\CXZ-WN-240312\Blue.tif'
band_name = 'Blue'
data = get_band(polygon_file_path,raster_file_path,band_name)
data

[[<function QgsFeature.id>, 0.022017243733406],
 [<function QgsFeature.id>, 0.022238849015319],
 [<function QgsFeature.id>, 0.024398337368453],
 [<function QgsFeature.id>, 0.021231108418305],
 [<function QgsFeature.id>, 0.021042457949683],
 [<function QgsFeature.id>, 0.021232169985968],
 [<function QgsFeature.id>, 0.018447675281756],
 [<function QgsFeature.id>, 0.017746535223851],
 [<function QgsFeature.id>, 0.022148178579514],
 [<function QgsFeature.id>, 0.020886332465501],
 [<function QgsFeature.id>, 0.020527766869597],
 [<function QgsFeature.id>, 0.018246426895468],
 [<function QgsFeature.id>, 0.018482152281555],
 [<function QgsFeature.id>, 0.020201297577345],
 [<function QgsFeature.id>, 0.017610908505673],
 [<function QgsFeature.id>, 0.01860836520666],
 [<function QgsFeature.id>, 0.016749428615167],
 [<function QgsFeature.id>, 0.014960615266512],
 [<function QgsFeature.id>, 0.01847354309045],
 [<function QgsFeature.id>, 0.014825766091478],
 [<function QgsFeature.id>, 0.015903058408